## Source: Wikipedia
## Retriever: Wikipedia Search
## Model: BioLinkBERT (PubMedQA)

In [1]:
import torch
from sklearn.metrics import classification_report, roc_auc_score

import sys
sys.path.append("../../") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc

In [2]:
from transformers import AutoTokenizer, BertForSequenceClassification, BertModel, DataCollatorWithPadding

model_name = "<path_to_bio-linkbert-large__pubmedqa_hf>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()
print("OK")

OK


In [3]:
def predict(question, passage):
    sequence = tokenizer.encode_plus(
        question, 
        passage, 
        return_tensors="pt",
        max_length=512,
        truncation=True
    )['input_ids']

    logits = model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = probabilities[2]
    
    return proba_yes

In [2]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

In [8]:
raw_df = pd.read_csv("../../../data/wikipedia_search_wiki/wikipedia_articles_keywords.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
)
print(len(df))
df.head(1)

1


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
184,dehumidifiers asthma,Asthma,Avoidance of triggers is a key component of im...,1.0,['asthma'],1.0,['dehumidifiers'],['dehumidifiers'],1.0,106.0,51,2019,0.0,Can dehumidifiers be used to control asthma?,dehumidifiers asthma


In [9]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (1 of 1) |##########################| Elapsed Time: 0:00:01 Time:  0:00:01


In [97]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,antibiotics common cold,Common cold,"The common cold, also known simply as a cold, ...",1.0,"['common', 'cold']",2.0,['antibiotic'],['antibiotic'],1.0,0.0,1_WH,wh_topics,0.0,Do antibiotics treat comon cold?,antibiotics common cold,0.244763


In [4]:
df.to_csv("predictions/biolinkbert_pubmedqa_wikipedia_search_wiki_preds_keywords.csv", index=0)

## Calc metrics

In [6]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [7]:
df = pd.read_csv("predictions/biolinkbert_pubmedqa_wikipedia_search_wiki_preds_keywords.csv")
print(len(df))
df.head(2)

513


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,cranberries urinary tract infections,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,['cranberry'],['cranberry'],1.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.574974
1,cranberries urinary tract infections,Urinary tract infection,The mainstay of treatment is antibiotics. Phen...,1.0,"['infection', 'urinary', 'tract']",3.0,['cranberry'],['cranberry'],1.0,81.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.183428


In [8]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [6]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(2)

539


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_h,health_belief,1.0,NaN,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate
538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13_h,health_belief,1.0,NaN,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe


In [10]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [11]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.6401,0.6640,0.9167,0.5982,0.6505
top1,0.5363,0.7584,0.8750,0.6875,0.6827
norm_linear,0.6332,0.6736,1.0000,0.6875,0.6751
norm_log,0.6125,0.6944,1.0000,0.6875,0.6862


## Question

In [12]:
raw_df = pd.read_csv("../../../data/wikipedia_search_wiki/wikipedia_articles_question.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
)
print(len(df))
df.head(1)

298


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
0,Can cranberries prevent urinary tract infections?,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,"['cranberry', 'prevent', 'can']","['cranberry', 'can']",2.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections


In [20]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (2 of 2) |##########################| Elapsed Time: 0:00:07 Time:  0:00:07


In [111]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,Is magnesium sulfate used to treat asthma?,Magnesium sulfate,The main external use is the formulation as ba...,1.0,"['magnesium', 'sulfate']",2.0,"['used', 'asthma', 'is', 'to', 'treat']","['to', 'asthma', 'is', 'used']",4.0,64.0,3_WH,wh_topics,0.0,Is magnesium sulfate used to treat asthma?,magnesium sulfate asthma,0.824242


In [112]:
df.to_csv("predictions/biolinkbert_pubmedqa_wikipedia_search_wiki_preds_question.csv", index=0)

## Calc Metrics

In [13]:
df = pd.read_csv("predictions/biolinkbert_pubmedqa_wikipedia_search_wiki_preds_question.csv")
print(len(df))
df.head(2)

298


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,Can cranberries prevent urinary tract infections?,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,"['cranberry', 'prevent', 'can']","['cranberry', 'can']",2.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.574974
1,Can cranberries prevent urinary tract infections?,Urinary tract infection,The mainstay of treatment is antibiotics. Phen...,1.0,"['infection', 'urinary', 'tract']",3.0,"['cranberry', 'prevent', 'can']","['cranberry', 'can']",2.0,81.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.183428


In [14]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(2)

354


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13_h,health_belief,1.0,NaN,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14_h,health_belief,1.0,NaN,NaN,0.5,Does listeria cause miscarriage?,listeria causes miscarriage


In [15]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [16]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.5882,0.6800,0.5417,0.4643,0.5632
top1,0.5363,0.6800,0.5417,0.4643,0.5683
norm_linear,0.5952,0.6816,0.5417,0.4643,0.5828
norm_log,0.5882,0.6800,0.5417,0.4464,0.5815
